In [37]:
!pip install rdflib

In [38]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/Структурирование и разметка данных'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [39]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL
from urllib.parse import quote
import os

In [40]:
graph = Graph()
graph.parse(path + "/dogs.ttl", format='turtle')
graph

<Graph identifier=N98781c9bf48c49b3b2bca70f81326bf0 (<class 'rdflib.graph.Graph'>)>

In [41]:
print(graph.serialize(format='ttl'))

@prefix : <http://www.semanticweb.org/ivanlyschev/ontologies/2024/8/dogs/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/ivanlyschev/ontologies/2024/8/dogs> a owl:Ontology .

:Corgi a owl:Class ;
    rdfs:subClassOf :Species .

:Description a owl:DatatypeProperty ;
    rdfs:domain :Image ;
    rdfs:range rdf:PlainLiteral .

:Doberman a owl:Class ;
    rdfs:subClassOf :Species .

:Labrador a owl:Class ;
    rdfs:subClassOf :Species .

:Pitbull a owl:Class ;
    rdfs:subClassOf :Species .

:Rottweiler a owl:Class ;
    rdfs:subClassOf :Species .

:Shepherd a owl:Class ;
    rdfs:subClassOf :Species .

:belongsToSpesies a owl:ObjectProperty ;
    rdfs:domain :Dogs ;
    rdfs:range :Species .

:doAction a owl:ObjectProperty ;
    rdfs:domain :Image ;
    rdfs:range :Action .

:hasAudio a owl:

In [42]:
URI = 'http://www.semanticweb.org/ivanlyschev/ontologies/2024/8/dogs/'
NS = Namespace(URI)

In [43]:
# Classes
dogs_class = NS.Dogs
audio_class = NS.Audio
action_class = NS.Action
species_class = NS.Species
image_class = NS.Image

# Species SubClasses
species_classes = {
    "corgi": NS.Corgi,
    "doberman": NS.Doberman,
    "labrador": NS.Labrador,
    "pitbull": NS.Pitbull,
    "rottweiler": NS.Rottweiler,
    "shepherd": NS.Shepherd
}

# Object and Data Properties
description_prop = NS.Description
belongs_to_species_prop = NS.belongsToSpecies
do_action_prop = NS.doAction
is_presented_prop = NS.isPresented
has_audio_prop = NS.hasAudio
is_carrying_prop = NS.isCarrying
is_attacking_prop = NS.isAttacking
is_standing_prop = NS.isStanding
is_sitting_prop = NS.isSitting
name_ru_prop = NS.nameRu
file_name_prop = NS.fileName

In [44]:
import pandas as pd


In [45]:
# Загрузка данных из dogs.csv
data_df = pd.read_csv(path + "/dogs.csv", delimiter=';')
for index, row in data_df.iterrows():
    dogs_instance_uri = URIRef(URI + row['classname'].replace(' ', '_'))
    species_class_uri = species_classes.get(row['classname'])

    # Добавление собаки
    graph.add((dogs_instance_uri, RDF.type, dogs_class))
    graph.add((dogs_instance_uri, belongs_to_species_prop, Literal(row['classname'])))
    graph.add((dogs_instance_uri, name_ru_prop, Literal(row['name (ru)'], lang='ru')))

    # Добавление описания внешности
    image_instance_uri = URIRef(URI + row['classname'].replace(' ', '_') + "_presented")
    graph.add((image_instance_uri, RDF.type, image_class))
    graph.add((image_instance_uri, description_prop, Literal(row['description'])))
    graph.add((dogs_instance_uri, is_presented_prop, image_instance_uri))

In [46]:
# Загрузка данных из dog_action.xlsx
actions_df = pd.read_csv(path + "/dogs_image_to_action.csv", delimiter=';')
for index, row in actions_df.iterrows():
    species_name  = row['file'].split('_')[0]
    dog_instance_uri = URIRef(URI + species_name)

    # Создание действия
    action_instance_uri = URIRef(URI + species_name + "_action" + str(index))
    graph.add((action_instance_uri, RDF.type, action_class))
    graph.add((action_instance_uri, is_attacking_prop, Literal(bool(row['isAttacking']))))
    graph.add((action_instance_uri, is_sitting_prop, Literal(bool(row['isSitting']))))
    graph.add((action_instance_uri, is_standing_prop, Literal(bool(row['isStanding']))))
    graph.add((action_instance_uri, file_name_prop, Literal(row['file'])))

    # Связывание действия с собакой
    graph.add((dog_instance_uri, do_action_prop, action_instance_uri))

In [47]:
from os.path import isfile, join

# Добавление данных о голосе из файлов в папке audio
audio_files = [f for f in os.listdir(path + "/audio") if isfile(join(path + "/audio", f))]
for audio_file in audio_files:

    species_name = os.path.splitext(audio_file)[0].split('_')[0]
    dogs_instance_uri = URIRef(URI + species_name)

    # Создание записи голоса
    audio_instance_uri = URIRef(URI + species_name + "_audio")
    graph.add((audio_instance_uri, RDF.type, audio_class))
    graph.add((dogs_instance_uri, has_audio_prop, audio_instance_uri))

In [48]:
graph.serialize(destination=path+"/UpdatedDogsOntology.ttl", format='turtle')

<Graph identifier=N98781c9bf48c49b3b2bca70f81326bf0 (<class 'rdflib.graph.Graph'>)>

In [49]:
!pip install tabulate
from tabulate import tabulate

In [50]:
# SPARQL-запрос
# Получение всех собак и их описаний
query = """
PREFIX ns: <http://www.semanticweb.org/ivanlyschev/ontologies/2024/8/dogs/>

SELECT ?dog ?nameRu ?description WHERE {
    ?dog rdf:type ns:Dogs .
    ?dog ns:nameRu ?nameRu .
    ?dog ns:isPresented ?presented .
    ?presented ns:Description ?description .
}
"""

# Выполнение запроса
results = graph.query(query)

table = []
for row in results:
    table.append([str(row.nameRu), str(row.description)])

# Использование tabulate для красивого вывода
headers = ["Имя", "Описание"]
print(tabulate(table, headers, tablefmt="grid"))

+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Имя       | Описание                                                                                                                                                                                                                                                                                                                   |
+===========+============================================================================================================================================================================================================================================================================================================================+
| Овчар

In [51]:
# Получение всех изображений собак
query = """
PREFIX ns: <http://www.semanticweb.org/ivanlyschev/ontologies/2024/8/dogs/>

SELECT ?file ?isAttacking ?isSitting ?isStanding WHERE {
    ?action rdf:type ns:Action .
    ?action ns:fileName ?file .
    OPTIONAL { ?action ns:isAttacking ?isAttacking . }
    OPTIONAL { ?action ns:isSitting ?isSitting . }
    OPTIONAL { ?action ns:isStanding ?isStanding . }
}
"""

# Выполнение запроса
results = graph.query(query)

table = []
for row in results:
    table.append([str(row.file), str(row.isAttacking), str(row.isSitting), str(row.isStanding)])

# tabulate для красивого вывода
headers = ["File", "IsAttacking", "Is Sitting", "Is Standing"]
print(tabulate(table, headers, tablefmt="grid"))

+-------------------+---------------+--------------+---------------+
| File              | IsAttacking   | Is Sitting   | Is Standing   |
+===================+===============+==============+===============+
| doberman_1.png    | true          | false        | false         |
+-------------------+---------------+--------------+---------------+
| doberman_2.webp   | false         | true         | false         |
+-------------------+---------------+--------------+---------------+
| doberman_3.jpg    | false         | false        | true          |
+-------------------+---------------+--------------+---------------+
| corgi_1.webp      | true          | false        | false         |
+-------------------+---------------+--------------+---------------+
| corgi_2.jpg       | false         | true         | false         |
+-------------------+---------------+--------------+---------------+
| corgi_3.jpg       | false         | false        | true          |
+-------------------+-------------

In [70]:
# Получение всех собак с породой
query = """
PREFIX ns: <http://www.semanticweb.org/ivanlyschev/ontologies/2024/8/dogs/>

SELECT ?dog ?nameRu ?species WHERE {
    ?dog rdf:type ns:Dogs .
    ?dog ns:nameRu ?nameRu .
    ?dog ns:belongsToSpecies ?species .
}
"""

# Выполнение запроса
results = graph.query(query)

table = []
for row in results:
    table.append([str(row.nameRu), str(row.species)])

# Использование tabulate для красивого вывода
headers = ["Name Ru", "Species"]
print(tabulate(table, headers, tablefmt="grid"))

+-----------+------------+
| Name Ru   | Species    |
+===========+============+
| Овчарка   | Shepherd   |
+-----------+------------+
| Доберман  | Doberman   |
+-----------+------------+
| Корги     | Corgi      |
+-----------+------------+
| Ротвейлер | Rottweiler |
+-----------+------------+
| Лабрадор  | Labrador   |
+-----------+------------+
| Питбуль   | PitBull    |
+-----------+------------+
